In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
from __future__ import unicode_literals

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import datetime

In [ ]:
calendar_history = pd.read_csv('data/calendar_data.csv', parse_dates = True)
calendar_history['start_date_time'] = pd.to_datetime(calendar_history['start_date_time'])
calendar_history['end_date_time'] = pd.to_datetime(calendar_history['end_date_time'])
calendar_history['created'] = pd.to_datetime(calendar_history['created'])
calendar_history['updated'] = pd.to_datetime(calendar_history['updated'])
calendar_history['recurring_event_id'] = calendar_history['recurring_event_id'].astype(str)

data_export_day = datetime.date(2016, 7, 13)
export_day = data_export_day.strftime('%Y-%m-%d') + ' 00:00:00'

print "All events: %s" % len(calendar_history)
calendar_history.tail()

In [ ]:
# remove weekends
calendar_history = calendar_history[calendar_history.start_date_time.dt.dayofweek < 5]
# remove tentative meetings
calendar_history = calendar_history[calendar_history.status != 'tentative']
# remove old cancelled meetings
calendar_data_filtered = calendar_history.drop(calendar_history[(calendar_history.status=='cancelled') 
                                                          & (calendar_history.updated < '2016-01-01 00:00:00')].index)
print "After filtering old cancelled items: %s" % len(calendar_data_filtered)
calendar_data_filtered = calendar_data_filtered.reset_index()
# calendar_data_filtered = calendar_data_filtered.sort_values(by=['event_id', 'conf'], ascending=True)
# calendar_data_filtered.to_csv('data/calendar_history_sorted.csv', sep=b',', encoding='utf-8', index=False)

def filter_cancelled_duplicates(group):
    if len(group[group.status == 'cancelled']) > 0 and len(group[group.status == 'confirmed']) > 0:
        return group[group.status == 'confirmed']
    if len(group[group.status == 'cancelled']) > 1:
        return group[group.status == 'cancelled'].iloc[[0]]
    return group

calendar_data_filtered = calendar_data_filtered.groupby(calendar_data_filtered.event_id, as_index=False).apply(filter_cancelled_duplicates)
# calendar_data_filtered = calendar_data_filtered.sort_values(by=['event_id', 'conf'], ascending=True)
print "After filtering event_id duplicates: %s" % len(calendar_data_filtered)

calendar_data_filtered.to_csv('data/calendar_data_filtered.csv', sep=b',', encoding='utf-8', index=False)

calendar_history_filtered = calendar_data_filtered[calendar_data_filtered.start_date_time < export_day]
calendar_history_filtered.to_csv('data/calendar_history_filtered.csv', sep=b',', encoding='utf-8', index=False)
calendar_history_filtered.tail()

In [ ]:
data_with_duration = calendar_data_filtered.copy()

def recurring_duration(row):
    if not row.recurring_event_id == 'nan':
        return row.end_date_time - row.start_date_time
    else:
        return pd.Timedelta(days=0)

def non_recurring_duration(row):
    if row.recurring_event_id == 'nan':
        return row.end_date_time - row.start_date_time
    else:
        return pd.Timedelta(days=0)

    
data_with_duration['recurring_events_duration'] = data_with_duration.apply(recurring_duration, axis=1)
data_with_duration['non_recurring_events_duration'] = data_with_duration.apply(non_recurring_duration, axis=1)
data_with_duration['total_duration'] = data_with_duration.end_date_time - data_with_duration.start_date_time

data_with_duration.tail()

In [ ]:
nearby_days = data_with_duration[data_with_duration.status == 'confirmed']
past_date = (data_export_day - datetime.timedelta(days=30)).strftime('%Y-%m-%d') + ' 00:00:00'
nearby_days = nearby_days[nearby_days.start_date_time > past_date]
# nearby_days = nearby_days[nearby_days.conf == 'Yellow']

nearby_agg_days = nearby_days.groupby(nearby_days['start_date_time'].dt.date).agg({'recurring_events_duration' : 'sum', 'non_recurring_events_duration' : 'sum'})

number_of_conf_rooms = len(nearby_days.conf.unique())
print "number_of_conf_rooms=%s" % number_of_conf_rooms

nearby_agg_days['non_recurring_events_duration'] = nearby_agg_days['non_recurring_events_duration'] / number_of_conf_rooms
nearby_agg_days['non_recurring_events_duration_in_hours'] = nearby_agg_days['non_recurring_events_duration'].dt.days * 24 + nearby_agg_days['non_recurring_events_duration'].dt.seconds / 3600
nearby_agg_days['recurring_events_duration'] = nearby_agg_days['recurring_events_duration'] / number_of_conf_rooms
nearby_agg_days['recurring_events_duration_in_hours'] = nearby_agg_days['recurring_events_duration'].dt.days * 24 + nearby_agg_days['recurring_events_duration'].dt.seconds / 3600
cols = nearby_agg_days.columns.tolist()
cols = cols[-1:] + cols[:-1]
nearby_agg_days = nearby_agg_days[cols]
nearby_agg_days.tail(30)

In [ ]:
plt.figure();

nearby_agg_days.plot.bar(stacked=True, figsize=(20,9), fontsize=12); 
plt.ylabel('Average usage per day [h]', fontsize=14)
plt.xlabel('Date', fontsize=14)
plt.title('Daily conference room usage per day', fontsize=16)
plt.gcf().subplots_adjust(bottom=0.20)
plt.gca().yaxis.grid(True)
plt.gca().set_yticks(np.arange(0, 9, 1))

#http://stackoverflow.com/questions/20394091/pandas-matplotlib-make-one-color-in-barplot-stand-out
plt.savefig('2_daily_usage_per_day.png')

In [ ]:
confirmed_events = data_with_duration[data_with_duration.status == 'confirmed']
confirmed_events = data_with_duration[data_with_duration.start_date_time < export_day]
conf_duration = confirmed_events.groupby('conf').agg({'recurring_events_duration' : 'sum', 'non_recurring_events_duration' : 'sum', 'total_duration' : 'sum', 'floor' : 'min'})

number_of_weekdays = len(confirmed_events["start_date_time"].dt.normalize().unique())
print "number_of_weekdays=%s" % number_of_weekdays

conf_duration['non_recurring_events_duration'] = conf_duration['non_recurring_events_duration'] / number_of_weekdays
conf_duration['non_recurring_events_duration_in_hours'] = conf_duration['non_recurring_events_duration'].dt.days * 24 + conf_duration['non_recurring_events_duration'].dt.seconds / 3600
conf_duration['recurring_events_duration'] = conf_duration['recurring_events_duration'] / number_of_weekdays
conf_duration['recurring_events_duration_in_hours'] = conf_duration['recurring_events_duration'].dt.days * 24 + conf_duration['recurring_events_duration'].dt.seconds / 3600

conf_duration = conf_duration.sort_values(by='total_duration', ascending=True)
conf_duration.index = conf_duration.floor + " " + conf_duration.index
cols = conf_duration.columns.tolist()
cols = cols[-1:] + cols[:-1]
conf_duration = conf_duration[cols]
conf_duration.tail()

In [ ]:
plt.figure();

conf_duration.plot.bar(stacked=True, figsize=(20,9), fontsize=12); 
plt.ylabel('Average usage per day [h]', fontsize=14)
plt.xlabel('Conference room', fontsize=14)
plt.title('Daily usage from last 90 days grouped by conference room', fontsize=16)
plt.gcf().subplots_adjust(bottom=0.25)
plt.gca().yaxis.grid(True)
plt.gca().set_yticks(np.arange(0, 11, 1))
plt.savefig('1_daily_usage_per_room.png')

In [ ]:
cancelation_period = calendar_history_filtered.copy()
cancelation_period.drop('organizer_name', axis=1, inplace=True)
cancelation_period.drop('organizer', axis=1, inplace=True)
cancelation_period.drop('creator', axis=1, inplace=True)
cancelation_period.drop('html_link', axis=1, inplace=True)
cancelation_period.drop('index', axis=1, inplace=True)
cancelation_period.drop('floor', axis=1, inplace=True)
cancelation_period.drop('recurring_event_id', axis=1, inplace=True)
# ignore private events
cancelation_period = cancelation_period[cancelation_period.created.notnull()]
# select cancelled events
cancelation_period = cancelation_period[cancelation_period.status == 'cancelled']
cancelation_period['cancelation_delay'] = cancelation_period.start_date_time - cancelation_period.updated
cancelation_period['cancelation_delay_in_h'] = cancelation_period['cancelation_delay'].dt.days * 24 + cancelation_period['cancelation_delay'].dt.seconds / 3600

# filter events cancelled in future
cancelation_period = cancelation_period[cancelation_period.cancelation_delay_in_h >= 0 ]
# filter events cancelled at least two week in advance
cancelation_period = cancelation_period[cancelation_period.cancelation_delay_in_h < 24 * 14 ]

cancelation_period['cancelation_delay_in_h'] = cancelation_period['cancelation_delay_in_h'].astype(int)
cancelation_period['cancelation_delay_in_h'] = -cancelation_period['cancelation_delay_in_h']

cancelation_cumulative_flow = cancelation_period.groupby(['cancelation_delay_in_h']).agg({'cancelation_delay_in_h': 'count'})

cancelation_cumulative_flow = cancelation_cumulative_flow.reindex(np.arange(cancelation_cumulative_flow.index.min(), cancelation_cumulative_flow.index.max() + 1)).fillna(0)
cancelation_cumulative_flow['cumulative_sum'] = cancelation_cumulative_flow.cancelation_delay_in_h.cumsum()
cancelation_cumulative_flow['cancelation_cumulative_percentage'] = 100*cancelation_cumulative_flow.cumulative_sum/cancelation_cumulative_flow.cancelation_delay_in_h.sum()

cancelation_cumulative_flow


In [ ]:
plt.figure();

cancelation_cumulative_diagram = cancelation_cumulative_flow.copy()
cancelation_cumulative_diagram.drop('cancelation_delay_in_h', axis=1, inplace=True)
cancelation_cumulative_diagram.drop('cumulative_sum', axis=1, inplace=True)
cancelation_cumulative_diagram.plot(figsize=(15,7), fontsize=10)

plt.legend(loc='best')    
plt.show()

In [ ]:
creation_period = calendar_history_filtered.copy()
creation_period.drop('organizer_name', axis=1, inplace=True)
creation_period.drop('creator', axis=1, inplace=True)
creation_period.drop('html_link', axis=1, inplace=True)
creation_period.drop('index', axis=1, inplace=True)
creation_period.drop('floor', axis=1, inplace=True)
# ignore private events
creation_period = creation_period[creation_period.created.notnull()]
# select confirmed events
creation_period = creation_period[creation_period.status == 'confirmed']
# ignore recurring events
creation_period = creation_period[creation_period.recurring_event_id == 'nan']

creation_period['creation_period'] = creation_period.start_date_time - creation_period.created
creation_period['creation_period_in_h'] = creation_period['creation_period'].dt.days * 24 + creation_period['creation_period'].dt.seconds / 3600

# ignore malformed events
creation_period = creation_period[creation_period.creation_period_in_h >= 0]

# filter events created at least two week in advance 
creation_period = creation_period[creation_period.creation_period_in_h < 24 * 14]

creation_period['creation_period_in_h'] = creation_period['creation_period_in_h'].astype(int)
creation_period['creation_period_in_h'] = -creation_period['creation_period_in_h']

creation_cumulative_flow = creation_period.groupby(['creation_period_in_h']).agg({'creation_period_in_h': 'count'})
creation_cumulative_flow = creation_cumulative_flow.reindex(np.arange(creation_cumulative_flow.index.min(), creation_cumulative_flow.index.max() + 1)).fillna(0)

creation_cumulative_flow['cumulative_sum'] = creation_cumulative_flow.creation_period_in_h.cumsum()
creation_cumulative_flow['creation_cumulative_percentage'] = 100*creation_cumulative_flow.cumulative_sum/creation_cumulative_flow.creation_period_in_h.sum()
creation_cumulative_flow

In [ ]:
plt.figure();

creation_cumulative_flow_diagram = creation_cumulative_flow.copy()
creation_cumulative_flow_diagram.drop('creation_period_in_h', axis=1, inplace=True)
creation_cumulative_flow_diagram.drop('cumulative_sum', axis=1, inplace=True)
creation_cumulative_flow_diagram.plot(figsize=(15,7), fontsize=10)

plt.legend(loc='best')    
plt.show()

In [ ]:
merged_period_df = pd.concat([creation_cumulative_flow_diagram, cancelation_cumulative_diagram], axis=1)

plt.figure();

merged_period_df.plot(figsize=(20,9), fontsize=12)

plt.gca().set_yticks(np.arange(0, 101, 10))
plt.xscale('symlog')
xticks = [-1, -2, -4, -8, -16, -24, -36, -48, -72, -96, -120, -144, -168, -192, -216, -240, -264, -288, -312]
xticks_labels = [-1, -2, -4, -8, -16, -24, -36, -48, -72, -96, -120, "", -168, "", -216, "", "", -288, ""]
plt.xticks(xticks, xticks_labels)
plt.ylabel('Percentage of all meetings of given type [%]', fontsize=14)
plt.xlabel('Period before the meeting [h]', fontsize=14)
plt.title('Cumulative flow diagram of meeting creation/cancellation', fontsize=16)
plt.gcf().subplots_adjust(bottom=0.1)
plt.legend(loc='lower right')
plt.grid(True)
plt.savefig('3_cancelation_duration.png')
plt.show()


In [ ]:
status_vs_recurrence = calendar_history_filtered.copy()
status_vs_recurrence['recurring'] = status_vs_recurrence.recurring_event_id != 'nan'
status_vs_recurrence_grouped = status_vs_recurrence.groupby(['recurring', 'status']).size()

count = len(status_vs_recurrence.index)

var_nonrecurring_cancelled = float(status_vs_recurrence_grouped[0][0])
var_nonrecurring_confirmed = float(status_vs_recurrence_grouped[0][1])
var_recurring_cancelled = float(status_vs_recurrence_grouped[1][0])
var_recurring_confirmed = float(status_vs_recurrence_grouped[1][1])

status_vs_recurrence_grouped

In [ ]:
labels = 'Non recurring & cancelled', 'Non recurring & confirmed', 'Recurring & cancelled', 'Recurring & confirmed'
sizes = [int(var_nonrecurring_cancelled/count*100), int(var_nonrecurring_confirmed/count*100), 
         int(var_recurring_cancelled/count*100), int(var_recurring_confirmed/count*100)]
colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral']

fig = plt.figure(1, figsize=(10,7))
ax = fig.add_subplot(111)
ax.axis('equal')
ax.pie(sizes, labels=labels, colors=colors,
        autopct='%1.1f%%', startangle=90)
plt.rcParams['font.size'] = 12.0
plt.savefig('4_status_vs_recurrence.png')
plt.show()


In [ ]:
valid_grouped_events = pd.read_csv('data/calendar_grouped_confirmed.csv', parse_dates = True)
valid_grouped_events['event_id'] = valid_grouped_events['event_id'].astype(str)
valid_grouped_events['recurring_event_id'] = valid_grouped_events['recurring_event_id'].astype(str)
valid_ids = pd.concat([valid_grouped_events['recurring_event_id'].dropna(), valid_grouped_events['event_id'].dropna()])

In [ ]:
recurring_events = calendar_history_filtered.copy()[calendar_history_filtered.recurring_event_id != 'nan']
valid_recurring_events = recurring_events[recurring_events['recurring_event_id'].isin(valid_ids)]

print "%s union %s = %s" % (len(recurring_events), len(valid_ids), len(valid_recurring_events))
valid_recurring_events

In [ ]:
grouped = valid_recurring_events.groupby(['recurring_event_id'])

often_cancelled_event_id_list = []
for name, group in grouped:
    if len(group[group.status == 'cancelled']) > len(group[group.status == 'confirmed']):
        often_cancelled_event_id_list.append(name)


In [ ]:
often_cancelled_events = valid_recurring_events[valid_recurring_events['recurring_event_id'].isin(often_cancelled_event_id_list)].copy()
often_cancelled_events.drop('index', axis=1, inplace=True)
often_cancelled_events['cancelation_delay'] = often_cancelled_events.start_date_time - often_cancelled_events.updated

often_cancelled_events.to_csv('data/often_cancelled_events.csv', sep=b',', encoding='utf-8', index=False)